+ 0.환경  
    + 1) 로컬 - pycharm  
    + 2) colab - gpu  
        
        
+ 1.데이터   
    + 1) 전부 올리기  
        + (1) 로컬   
        + (2) colab  
            
    + 2) 폴더에서 가져오기  
            
              
            
              
  
+ 2. 모델  
    + 1) 기본 cnn  
    + 2) Vgg 16   
    + 3) Inception v3   
    + 4) Resnet50  
            
              
  
+ 3. 역전파  
    + 1) optimizer  
    + 2) epochs / batch_size  
    + 3) callback
            
              
  
+ 4. report  
            
              
  
+ 5. 추후 점수 향상 방법
    + 1) 사진 크롭  
    + 2) 사진 gray 처리 
    + 3) 사진 비틀기
    

# 0.환경  

 1) 로컬 - pycharm  
   
   
- 장점 : 사진을 끌어오는 것이 빠르다
- 단점 ; gpu가 없을 시 학습이 너무 느리다
    

 2) colab - gpu
   
   
- 장점 : gpu를 통해 다양한 학습을 반복시키는 것이 빠르다
- 단점 ; 사진을 끌어오는것이 너무 느리다
         하지만 한번 전부 읽어온다면 다음부터는 조금더 시간이 적게 든다
         - 폴더를 통해 가져오는 시간이 너무 느리다
         

In [ ]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense,Dropout,MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

# 1.데이터 
- x, y 모두 np 사용

##         1) 전부 올리기

In [ ]:
# (1) 로컬에서

# csv

import numpy as np
csv_data=np.loadtxt("./train/train.tsv",delimiter="\t",dtype='str')



# image

import PIL.Image as pilimg
pix=[]
for i in range(len(y_plant)):
    im = pilimg.open( "./train/"+csv_data[i][0] )
    #pix = np.r_[[pix],[np.array(im)]]
    pix.append(np.array(im))
    if i%100==0:
        print(i)


In [ ]:
# (2) colab 에서

# csv
from google.colab import drive
import numpy as np

drive.mount('/content/gdrive')

csv_data = np.loadtxt("/content/gdrive/My Drive/~~~/filename.tsv",delimiter="\t",dtype='str')


#image
import PIL.Image as pilimg

pix=[]
for i in range(len(y_plant)):
    im = pilimg.open( "/content/gdrive/My Drive/~~~/"+csv_data[i][0] )
    pix.append(np.array(im))
    
    if i%500==0:
        print(i)


In [ ]:
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(pix, Y_data, test_size = 0.1, random_state=11)


# Without data augmentation i obtained an accuracy of 0.98114
history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, Y_val), verbose = 2)

## 2) 폴더로 올리기

In [ ]:

train_data_dir='/content/gdrive/My Drive/plant_disease_data/train_folder'
img_height=256
img_width=256
epochs = 10
batch_size = 100

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


history=model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = epochs)

# 2.모델

## 1) 기본 CNN

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(20, activation='softmax')) # 2 because we have cat and dog classes

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()



## 2) VGG 16  CNN으로 만들기

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model=Sequential()
#256 244
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(256,256,3),activation='relu'))
model.add(MaxPool2D((2,2),strides=(2,2),padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPool2D((2,2),strides=(2,2),padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPool2D((2,2),strides=(2,2),padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPool2D((2,2),strides=(2,2),padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(filters=1024,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPool2D((2,2),strides=(2,2),padding='valid'))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(20,activation='softmax'))

model.summary()



## 3) inception v3

In [ ]:

input = Input(shape=(224, 224, 3))

model=tf.keras.applications.InceptionV3(
        include_top=False,
        weights='imagenet',
        input_tensor=input,
        input_shape=(256,256,3),
        pooling=None,
        #classes=20,
        #classifier_activation="softmax",
    )

x = model.output
x = tf.keras.layers.Flatten()(x)
x = Dense(1024, name='fully', kernel_initializer='random_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(512, kernel_initializer='random_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(20, activation='softmax', name='softmax')(x)

model = Model(model.input, x)
model.summary()

## 4) resnet50

In [ ]:

from keras.applications import ResNet50

input = Input(shape=(224, 224, 3))

model = ResNet50(input_tensor=input, include_top=False, weights=None, pooling='max')

x = model.output
x = tf.keras.layers.Flatten()(x)
x = Dense(1024, name='fully', kernel_initializer='random_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(512, kernel_initializer='random_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(20, activation='softmax', name='softmax')(x)

model = Model(model.input, x)
model.summary()

### 모델저장 /불러오기

In [ ]:
from keras.models import load_model

#저장
model.save('/content/gdrive/My Drive/resnet50a.h5')

In [ ]:
#불러오기
model = load_model('/content/gdrive/My Drive/resnet50.h5')

# 3. 역전파

In [ ]:
# 1)
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# 2)
epochs = 25 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 100

# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

# 3)Callback     Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)



In [ ]:
# Without data augmentation i obtained an accuracy of 0.98114
history = model.fit(X_train, Y_train, 
                    batch_size = batch_size, 
                    epochs = epochs, 
                    validation_data = (X_val, Y_val), 
                    verbose = 2,
                    callbacks=[learning_rate_reduction]
                   )

### callback도 중요한듯!

In [ ]:
##### 빨리 끝낼수 있는 callback
early_stopping = EarlyStopping()
model.fit(X_train, Y_train, nb_epoch= 1000, callbacks=[early_stopping])

# OR

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(weightpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')



# 4. Report

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(20)) 

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(Y_true, Y_pred_classes)
print(report)

In [ ]:
# Display some error results 

# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((224,224,3)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors[-6:]

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

# 4. 추후 점수 향상 방법

## 1) 사진크롭

## 2) 사진 흑백화

## 3) 사진 비틀기

In [ ]:
# # With data augmentation to prevent overfitting (accuracy 0.99286)

# datagen = ImageDataGenerator(
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#         zoom_range = 0.1, # Randomly zoom image 
#         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=False,  # randomly flip images
#         vertical_flip=False)  # randomly flip images


# datagen.fit(X_train)

In [ ]:
# data_gen_train = ImageDataGenerator(rescale=1/255.)

# data_gen_valid = ImageDataGenerator(rescale=1/255.)

# train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

# valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

# model.fit(train_generator, epochs=2, validation_data=valid_generator).

In [ ]:
# # Fit the model
# history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
#                               epochs = epochs, 
#                               validation_data = (X_val,Y_val),
#                               verbose = 2, 
#                               steps_per_epoch=X_train.shape[0] // batch_size,
#                               callbacks=[learning_rate_reduction])